In [1]:
ENV_TYPE = "TEST"

if(ENV_TYPE != "TEST"):
  !git clone "https://github.com/kk-digital/kcg-ml-sd1p4.git"
  %cd kcg-ml-sd1p4
  !pip3 install -r requirements.txt
  exit()
  base_directory = "./"
else:
  base_directory = "../"

# Magical check for fixing all of our directory issues
import subprocess
output = subprocess.check_output(["pwd"], universal_newlines=True)
if "notebooks" in output:
    %cd ..
del output

/Users/kenjehofilena/Desktop/Work/GoWorkSpace/src/github.com/kk-digital/kcg-ml-sd1p4


In [2]:
!python3 ./download_models.py

Creating directory tree folders.                                                        
	INFO: All model paths have been created.
Creating directory tree folders....[DONE]	0.74ms                                  
INFO: Downloading models. This may take a while.
Checking if minio server is accessible...
Minio server is not accessible...


In [ ]:
!python3 ./process_models.py

Some weights of the model checkpoint at ./input/model/clip/vit-large-patch14/vit-large-patch14.safetensors were not used when initializing CLIPVisionModelWithProjection: ['text_model.encoder.layers.4.mlp.fc2.weight', 'text_model.encoder.layers.10.mlp.fc2.weight', 'text_model.encoder.layers.8.self_attn.q_proj.bias', 'text_model.encoder.layers.2.self_attn.k_proj.weight', 'text_model.encoder.layers.6.self_attn.out_proj.bias', 'text_model.encoder.layers.10.layer_norm1.bias', 'text_model.encoder.layers.9.self_attn.out_proj.bias', 'text_model.encoder.layers.5.mlp.fc2.bias', 'text_model.encoder.layers.9.layer_norm2.bias', 'text_model.encoder.layers.5.self_attn.k_proj.weight', 'text_model.encoder.layers.1.mlp.fc1.weight', 'text_model.encoder.layers.11.self_attn.q_proj.weight', 'text_model.encoder.layers.3.mlp.fc1.weight', 'text_model.encoder.layers.9.layer_norm1.weight', 'text_model.encoder.layers.2.self_attn.q_proj.weight', 'text_model.encoder.layers.8.self_attn.out_proj.bias', 'text_model.en

In [ ]:
import torch
import clip
import os
import sys
import torch
import torchvision.transforms as T
import time
import shutil
from os.path import join
from torchvision.transforms import ToPILImage

sys.path.insert(0, base_directory)

output_base_dir = join(base_directory, "./output/sd2-notebook/")
output_directory = join(output_base_dir, "clip_image_encoder/")

try:
    shutil.rmtree(output_directory)
except Exception as e:
    print(e, "\n", "Creating the path...")
    os.makedirs(output_directory, exist_ok=True)
else:
    os.makedirs(output_directory, exist_ok=True)

from stable_diffusion.stable_diffusion import StableDiffusion
from stable_diffusion.model.clip_image_encoder import CLIPImageEncoder
from stable_diffusion.model.clip_text_embedder import CLIPTextEmbedder
from transformers import CLIPTextModel, CLIPTokenizer
from utility.labml.monit import section
# from stable_diffusion.utils.utils import SectionManager as section
from stable_diffusion.utils_model import *
from stable_diffusion.utils_backend import *
from stable_diffusion.model_paths import IODirectoryTree
from configs.model_config import ModelPathConfig
from pathlib import Path

In [ ]:
# Setup config (paths and such)

base_dir = os.getcwd()
sys.path.insert(0, base_dir)

batch_size = 1
model_config = ModelPathConfig()
pt = IODirectoryTree(model_config)

In [ ]:
# Initialize clip
clip_text_embedder = CLIPTextEmbedder(device=get_device())
clip_text_embedder.load_submodels()
get_memory_status(device=get_device())

In [ ]:
# get the text embeddings
null_prompt = torch.tensor(()).to(get_device())
embeddings = clip_text_embedder.forward("A computer virus dancing tango.")
get_memory_status(device=get_device())

In [ ]:
# check their shape
null_prompt.shape, embeddings.shape

In [ ]:
# Cleanup clip
del clip_text_embedder

In [ ]:
# save them to disk
torch.save(embeddings, join(output_directory, "embeddings.pt"))
torch.save(null_prompt, join(output_directory, "null_prompt.pt"))

In [ ]:
# Load Stable Diffusion
sd = StableDiffusion(device=get_device(), n_steps=25)
sd.quick_initialize().load_autoencoder().load_decoder()
sd.model.load_unet()

In [ ]:
images = sd.generate_images_latent_from_embeddings(null_prompt = null_prompt, embedded_prompt=embeddings, batch_size = batch_size)
get_memory_status(get_device())

In [ ]:
images.shape

In [ ]:
from PIL import Image
from stable_diffusion.utils_image import to_pil
if batch_size > 1:
    grid = torchvision.utils.make_grid(images, nrow=2, normalize=False, range=(-1, 1))
    img = to_pil(grid)
else:
    img = to_pil(images[0].squeeze())
img

In [ ]:
images.shape

In [ ]:
img_encoder = CLIPImageEncoder(device=get_device())
get_memory_status(device = get_device())

In [ ]:
img_encoder.load_submodels()
get_memory_status(device = get_device())
img_encoder.initialize_preprocessor(do_center_crop=True)

In [ ]:
img_encoder.image_processor

In [ ]:
prep_from_img = img_encoder.preprocess_input(img)
type(img)

In [ ]:
img_encoder.image_processor

In [ ]:
prep_from_tensor = img_encoder.preprocess_input(images)
type(images)

In [ ]:
prep_from_img.shape, prep_from_tensor.shape

In [ ]:
torch.all(prep_from_img.to(get_device()) == prep_from_tensor)

In [ ]:
torch.norm(prep_from_img.to(get_device()) - prep_from_tensor)

In [ ]:
to_pil(prep_from_img.squeeze(0))

In [ ]:
to_pil(prep_from_tensor.squeeze(0))

In [ ]:
to_pil((prep_from_img - prep_from_tensor).squeeze(0))

In [ ]:
if batch_size > 1:
    grid = torchvision.utils.make_grid([prep_from_img, prep_from_tensor], nrow=2, normalize=False, range=(-1, 1))
    img = to_pil(grid)
else:
    img = to_pil(prep_from_img.squeeze(0))
img

In [ ]:
import torchvision
grid = torchvision.utils.make_grid([prep_from_img.squeeze(), prep_from_tensor.squeeze()], nrow=2, normalize=False, range=(-1, 1))
img = to_pil(grid)